In [ ]:
import os
import math
import librosa
import numpy as np
import tensorflow.keras as keras
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow import losses
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Flatten,Dense,MaxPooling2D
from scipy.stats import reciprocal
from sklearn.model_selection import RandomizedSearchCV
from sklearn.preprocessing import StandardScaler


In [ ]:
DATASET_PATH = # Enter File Path Here
SAMPLE_RATE = 44100 # For this project sampling rate is 44100 Hz and 20 sec audio used. 
TRACK_DURATION = 20

In [ ]:
data_test = {
    "mfcc": [],
    }
data = {
    "labels": [],
    "mfcc": [],
}

In [ ]:
for dirpath, dirnames,filenames in os.walk(DATASET_PATH):
    if dirpath is not DATASET_PATH:
        
        if dirpath ==: # Enter train data path
            for f in filenames:
                file_path = os.path.join(dirpath,f)
                signal, sample_rate = librosa.load(file_path, sr = SAMPLE_RATE, duration = 20) 
                mfcc = librosa.feature.mfcc(signal,sample_rate, n_mfcc = 50,n_fft = 2048,hop_length = 1024)
                mfcc = mfcc.T
                
                data["mfcc"].append(mfcc)
                
                if f[0] == "F":
                    data["labels"].append(0)
                elif f[0] == "H":
                    data["labels"].append(1)
        elif dirpath == : # Enter test datapath
            for f in filenames:
                file_path = os.path.join(dirpath,f)
                signal, sample_rate = librosa.load(file_path, sr = SAMPLE_RATE, duration = 20)    
                mfcc = librosa.feature.mfcc(signal,sample_rate, n_mfcc = 50,n_fft = 2048,hop_length = 1024)
                mfcc = mfcc.T
                data_test["mfcc"].append(mfcc)
                        
                        
            
        

In [ ]:
mfcc_train = np.array(data["mfcc"])
y = np.array(data["labels"])
mfcc_test = np.array(data_test["mfcc"])

In [ ]:
scalers = {} # Scaling Train and Test Data
scalers1 = {}

for i in range(mfcc_train.shape[1]):
    scalers[i] = StandardScaler()
    mfcc_train[:,i,:] = scalers[i].fit_transform(mfcc_train[:, i, :])
for i in range(mfcc_test.shape[1]):
    mfcc_test[:, i, :] = scalers[i].transform(mfcc_test[:, i, :])

for i in range(mfcc_train.shape[2]):
    scalers1[i] = StandardScaler()
    mfcc_train[:,:,i] = scalers1[i].fit_transform(mfcc_train[:, :, i])
for i in range(mfcc_test.shape[2]):
    mfcc_test[:, :, i] = scalers1[i].transform(mfcc_test[:, :, i])
    

In [ ]:
mfcc_train = mfcc_train[..., np.newaxis] # Add new axis to apply 2-D CNN
mfcc_test = mfcc_test[..., np.newaxis]

In [ ]:
mfcc_train,mfcc_valid,y_train,y_valid=train_test_split(mfcc_train,y,test_size = 0.2,shuffle = True)

In [ ]:
optimizer = keras.optimizers.SGD(lr = 0.163430258482965 )

In [ ]:
model = keras.Sequential()


model.add(keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=(mfcc_train.shape[1],mfcc_train.shape[2],1)))
model.add(keras.layers.MaxPooling2D((3, 3), strides=(2, 2), padding='same'))
model.add(keras.layers.BatchNormalization())

model.add(keras.layers.Conv2D(32, (3, 3), activation='relu'))
model.add(keras.layers.MaxPooling2D((3, 3), strides=(2, 2), padding='same'))
model.add(keras.layers.BatchNormalization())

model.add(keras.layers.Conv2D(32, (2, 2), activation='relu'))
model.add(keras.layers.MaxPooling2D((2, 2), strides=(2, 2), padding='same'))
model.add(keras.layers.BatchNormalization())


model.add(keras.layers.Flatten())
model.add(keras.layers.Dense(128, activation='relu'))
model.add(keras.layers.Dense(128, activation='relu'))

model.add(keras.layers.Dense(1, activation='sigmoid'))

In [ ]:
model.compile(optimizer=optimizer,
                  loss=tf.keras.losses.BinaryCrossentropy(from_logits=True))

In [ ]:
# Since it is cnn long training needed to increase accuracy.
history = model.fit(mfcc_train, y_train,batch_size=32, epochs=2000,validation_data =[mfcc_valid,y_valid] ,callbacks = [keras.callbacks.EarlyStopping(patience = 100)])

In [ ]:
y_pred = model.predict(mfcc_test)

In [ ]:
np.round(y_pred)

In [ ]:
# Trying other values which is given healty values outside train and test set

data_path= # This is for test the result
signal2, sample_rate = librosa.load(data_path, sr = 44100, duration = 20)
mfccx = librosa.feature.mfcc(signal2,44100, n_mfcc = 50,n_fft = 2048,hop_length = 1024)
mfccx = mfccx.T
mfccx = mfccx[np.newaxis,...]
for i in range(mfccx.shape[2]):
    mfccx[:, :, i] = scalers1[i].transform(mfccx[:, :, i])
for i in range(mfcc_test.shape[1]):
    mfccx[:, i, :] = scalers[i].transform(mfccx[:, i, :])
mfccx = mfccx[..., np.newaxis]

In [ ]:
y_pred = model.predict(mfccx)

In [ ]:
y_pred

In [ ]:
data_path=# This is for test the result
signal2, sample_rate = librosa.load(data_path, sr = 44100, duration = 20)
mfccx2 = librosa.feature.mfcc(signal2,44100, n_mfcc = 50,n_fft = 2048,hop_length = 1024)
mfccx2 = mfccx2.T
mfccx2 = mfccx2[np.newaxis,...]
for i in range(mfccx2.shape[2]):
    mfccx2[:, :, i] = scalers1[i].transform(mfccx2[:, :, i])
for i in range(mfcc_test.shape[1]):
    mfccx2[:, i, :] = scalers[i].transform(mfccx2[:, i, :])
mfccx2 = mfccx2[..., np.newaxis]


In [ ]:
y_pred = model.predict(mfccx2)

In [ ]:
y_pred

In [ ]:
# Determining neural network paramters with RandomSearch
def build_model(n_hidden = 1,n_neurons = 30, learning_rate = 3e-3, input_shape = (mfcc_train.shape[1],mfcc_train.shape[2],1)):
    model = keras.models.Sequential()
    model.add(keras.layers.Conv2D(32, (3,3), activation='relu', input_shape=input_shape))
    model.add(keras.layers.MaxPooling2D((3,3), strides=(2,2), padding='same'))
    model.add(keras.layers.BatchNormalization())
    
    model.add(keras.layers.Conv2D(32, (3,3), activation='relu'))
    model.add(keras.layers.MaxPooling2D((3,3), strides=(2,2), padding='same'))
    model.add(keras.layers.BatchNormalization())
    
    model.add(keras.layers.Conv2D(32, (3,3), activation='relu'))
    model.add(keras.layers.MaxPooling2D((3,3), strides=(2,2), padding='same'))
    model.add(keras.layers.BatchNormalization())
    
    
    model.add(keras.layers.Flatten())            
    model.add(keras.layers.InputLayer())
    for layer in range(n_hidden):
        model.add(keras.layers.Dense(n_neurons,activation = "relu"))
    model.add(keras.layers.Dense(1,activation = "sigmoid"))
    optimizer = keras.optimizers.SGD(lr = learning_rate)
    model.compile(loss = tf.keras.losses.BinaryCrossentropy(from_logits=True),optimizer = optimizer)
    return model


xxx = keras.wrappers.scikit_learn.KerasRegressor(build_model)

param_distribs = {
    "n_hidden":[1,2,3,4],
    "n_neurons": np.arange(1,200),
    "learning_rate": reciprocal(3e-3,3e-1),
}

rnd_search_cv = RandomizedSearchCV(xxx,param_distribs,n_iter = 20)
rnd_search_cv.fit(mfcc_train,y_train,epochs = 100,batch_size = 32,validation_data =[mfcc_valid,y_valid] ,callbacks = [keras.callbacks.EarlyStopping(patience = 10)])